In [2]:
%%writefile requirements.txt
torch==2.5.0
torchvision==0.20.0
onnx==1.17.0
onnxruntime-gpu==1.19.2
tqdm==4.67.1
albumentations==1.4.20
pandas==2.2.3
seaborn==0.13.2

Writing requirements.txt


In [3]:

!cat requirements.txt



torch==2.5.0
torchvision==0.20.0
onnx==1.17.0
onnxruntime-gpu==1.19.2          
tqdm==4.67.1
albumentations==1.4.20           
pandas==2.2.3
seaborn==0.13.2                  
